# Libraries and modules


In [2]:
import pandas as pd 
import os 
import datetime
from ML.frontpage import frontpage as fp, regexdata as rd
from ML.directories import master_directory as md 
from ML.generate_url import returnUrlHousingtoScrap

from ML.tools import savedata as sd
#Visual de data (permite ver todo un df)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# __ENVIRONMENT__

In [7]:
# Variables 

mth = 'FEB'       #Update each month
yr = '2024'       #Update each month

op = 'Rent'
web = 'ML'

#Master directory
master_dir = md()
master_dir = f'{master_dir}/{web}'


#Directories to export files 
result_dir = f'{master_dir}/FRONTPAGE/JOIN'
monitoring_dir = f'{master_dir}/FRONTPAGE/MONITORING/{yr}/{mth}'

#Export files 
###############################################################################################################
#This files is going to be joined for create the master file, which contains all the data for the fron page
fl_pre = f'{web}_join_{op}_{mth}{yr}_'

fl_master = f'{web}_{mth}{yr}_{op}_Front'


# Generate links 

In [8]:
df_enlaces = returnUrlHousingtoScrap(op)


Usted generó 330 enlaces principales de portada para arriendo


# Split data 

In [4]:
#
num_dfs = len(df_enlaces) // 24 + (1 if len(df_enlaces) % 24 != 0 else 0)
# Dividir df_enlaces entre 5000 observaciones cada uno para hacer el scrapign sin problemas
dfs = {f'{fl_pre}{i}': df_enlaces.iloc[i:i+24] for i in range(0, len(df_enlaces), 24)}
# Nombres de df_enlaces
print(f'Se crearon {num_dfs} DataFrames para df_enlaces:')
for key in dfs:
    print(f'{key}')

Se crearon 2 DataFrames para df_enlaces:
ML_join_Rental_FEB2024_0
ML_join_Rental_FEB2024_24


# __Scraping__

In [5]:
try:
    start_time = datetime.datetime.now()
    print(f"El proceso de scraping ha comenzado a las {start_time.strftime('%H:%M')} del {start_time.strftime('%d/%m/%Y')}")
    
    for df_key, df_value in dfs.items():
        print(f"Realizando scraping en {df_key}")
        global resultado_df
        # Inicializa un DataFrame vacío para almacenar los resultados
        resultado_df = pd.DataFrame()
        contador_actual = 0
        # Inicializa una lista para almacenar los resultados de cada enlace
        resultados = []
        # Itera a través de la lista de enlaces
        for index, row in df_value.iterrows():
            # Inicializar contador de enlaces procesados en la iteración actual
            contador_actual += 1
            print(f"Enlace {contador_actual} de {len(df_value)}")
            
            link = row["Enlace"]
            df_resultado = fp(link)  # Cambia el valor de 'pages' según tus necesidades
            resultados.append(df_resultado)

        # Concatena los resultados en un solo DataFrame
        resultado_df = pd.concat(resultados, ignore_index=True)
        
        # sd -> save_to_csv()
        sd(resultado_df, f'{result_dir}/{df_key}')

        print(f"Scraping completado para {df_key}.")
        print("--------------------------------------------------")
    #Fin del scraping: 
    end_time = datetime.datetime.now()
    print(f"El proceso de scraping ha terminado a las {end_time.strftime('%H:%M')} del {end_time.strftime('%d/%m/%Y')}")
    
except Exception as e:
    print(f"Se produjo un error en el scraping: {e}")

El proceso de scraping ha comenzado a las 16:07 del 08/02/2024
Realizando scraping en ML_join_Rental_FEB2024_0
Enlace 1 de 24
Enlace 2 de 24
Enlace 3 de 24
Enlace 4 de 24
Enlace 5 de 24
Enlace 6 de 24
Enlace 7 de 24
Enlace 8 de 24
Enlace 9 de 24
Enlace 10 de 24
Enlace 11 de 24
Enlace 12 de 24
Enlace 13 de 24
Enlace 14 de 24
Enlace 15 de 24
Enlace 16 de 24
Enlace 17 de 24
Enlace 18 de 24
Enlace 19 de 24
Enlace 20 de 24
Enlace 21 de 24
Enlace 22 de 24
Enlace 23 de 24
Enlace 24 de 24
Scraping completado para ML_join_Rental_FEB2024_0.
--------------------------------------------------
Realizando scraping en ML_join_Rental_FEB2024_24
Enlace 1 de 24
Enlace 2 de 24
Enlace 3 de 24
Enlace 4 de 24
Enlace 5 de 24
Enlace 6 de 24
Enlace 7 de 24
Enlace 8 de 24
Enlace 9 de 24
Enlace 10 de 24
Enlace 11 de 24
Enlace 12 de 24
Enlace 13 de 24
Enlace 14 de 24
Enlace 15 de 24
Enlace 16 de 24
Enlace 17 de 24
Enlace 18 de 24
Enlace 19 de 24
Enlace 20 de 24
Enlace 21 de 24
Enlace 22 de 24
Enlace 23 de 24
Enla

## Join data 

In [6]:
# Obtén la lista de nombres de archivo CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(result_dir) if archivo.endswith('.csv') and archivo.startswith(fl_pre)]

# Inicializa una lista para almacenar los DataFrames cargados desde los archivos CSV
dataframes = []

# Carga cada archivo CSV en un DataFrame y agrega a la lista
for archivo in archivos_csv:
    ruta_completa = os.path.join(result_dir, archivo)
    df = pd.read_csv(ruta_completa, sep=';', encoding='utf-16')
    dataframes.append(df)
    

# Combina los DataFrames en uno solo
portadas = pd.concat(dataframes, ignore_index=True)
exportable = rd(portadas)
print(len(exportable))

C:\Users\luisG\GIT\SCRAPING\SCRAPERS\BUILDINGS\ML\ML\frontpage.py:641: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio'] = df['Precio'].str.replace(".", "")


6554


## Export the master file

In [7]:
sd(exportable, f'{monitoring_dir}/{fl_master}')